This notebook will go through how to access and run code on youtube thumbnails



In [ ]:
%pip install cmake
%pip install requests


In [ ]:
import os
print(os.listdir())


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Path to the thumbnails folder
thumbnails_folder = r'C:\Users\alexa\Thesis\complete thumbnails\thumbnails'


# List all files in the thumbnails folder
files = os.listdir(thumbnails_folder)

# Filter out non-image files (optional) and select 3 images
image_files = [f for f in files if f.endswith(('.png', '.jpg', '.jpeg'))][:3]

# Display the selected 3 images
plt.figure(figsize=(10, 10))

for i, image_file in enumerate(image_files):
    image_path = os.path.join(thumbnails_folder, image_file)
    image = Image.open(image_path)

    plt.subplot(1, 3, i + 1)
    plt.imshow(image)
    plt.title(image_file)
    plt.axis('off')

plt.show()


In [ ]:
%pip install pytube opencv-python
%pip install dlib imutils



In [ ]:
# Import dependencies
import cv2
import numpy as np
from PIL import Image
import io
import time
import base64
import html


## Haar Cascade Classifier
For this tutorial we will run a simple object detection algorithm called Haar Cascade on our images and video fetched from our webcam. OpenCV has a pre-trained Haar Cascade face detection model.

In [ ]:

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

Test Code for 3 Images


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import dlib
import pandas as pd

# Path to the thumbnails folder
thumbnails_folder = r'C:\Users\alexa\Thesis\complete thumbnails\thumbnails'

# Load the pre-trained dlib model for facial landmarks detection
predictor_path = r"C:\Users\alexa\Thesis\shape_predictor_68_face_landmarks.dat\shape_predictor_68_face_landmarks.dat"
face_landmark_predictor = dlib.shape_predictor(predictor_path)

# Initialize dlib's face detector
detector = dlib.get_frontal_face_detector()

# Define AU weights for each emotion to compute confidence levels
emotion_au_weights = {
    'Happiness': {'AU12': 0.5, 'AU6': 0.5},
    'Sadness': {'AU1': 0.3, 'AU4': 0.4, 'AU15': 0.3},
    'Surprise': {'AU5': 0.5, 'AU26': 0.5},
    'Disgust': {'AU9': 0.5, 'AU10': 0.5},
    'Fear': {'AU4': 0.3, 'AU5': 0.3, 'AU7': 0.4},
    'Anger': {'AU4': 0.5, 'AU7': 0.5},
    'Neutral': {}
}

# Function to compute AUs based on landmarks
def compute_facs_aus(landmarks):
    # Extract key facial coordinates
    left_eye_y = np.mean([landmarks.part(i).y for i in range(36, 42)])  # Left eye
    right_eye_y = np.mean([landmarks.part(i).y for i in range(42, 48)])  # Right eye
    mouth_top_y = np.mean([landmarks.part(i).y for i in range(48, 55)])  # Top of mouth
    mouth_bottom_y = np.mean([landmarks.part(i).y for i in range(55, 60)])  # Bottom of mouth
    left_eyebrow_y = np.mean([landmarks.part(i).y for i in range(17, 22)])  # Left eyebrow
    right_eyebrow_y = np.mean([landmarks.part(i).y for i in range(22, 27)])  # Right eyebrow
    nose_tip_y = landmarks.part(30).y  # Nose tip
    upper_lip_y = landmarks.part(62).y  # Upper lip (central)
    lower_lip_y = landmarks.part(66).y  # Lower lip (central)

    mouth_corner_left = landmarks.part(48).y  # Left corner of mouth
    mouth_corner_right = landmarks.part(54).y  # Right corner of mouth

    # Define AUs based on specific facial regions
    aus = {
        'AU1': 'Inner Brow Raiser' if left_eyebrow_y < left_eye_y else 'None',
        'AU2': 'Outer Brow Raiser' if left_eyebrow_y < left_eye_y - 5 else 'None',
        'AU4': 'Brow Lowerer' if left_eyebrow_y > left_eye_y else 'None',
        'AU5': 'Upper Lid Raiser' if left_eye_y < nose_tip_y else 'None',
        'AU6': 'Cheek Raiser' if mouth_corner_left < mouth_top_y and mouth_corner_right < mouth_top_y else 'None',
        'AU7': 'Lid Tightener' if left_eye_y > nose_tip_y else 'None',
        'AU9': 'Nose Wrinkler' if nose_tip_y < upper_lip_y else 'None',
        'AU10': 'Upper Lip Raiser' if upper_lip_y < lower_lip_y else 'None',
        'AU12': 'Lip Corner Puller' if mouth_corner_left < mouth_top_y and mouth_corner_right < mouth_top_y else 'None',
        'AU14': 'Dimpler' if mouth_corner_left > mouth_top_y and mouth_corner_right > mouth_top_y else 'None',
        'AU15': 'Lip Corner Depressor' if mouth_corner_left > upper_lip_y and mouth_corner_right > upper_lip_y else 'None',
        'AU17': 'Chin Raiser' if lower_lip_y > upper_lip_y else 'None',
        'AU20': 'Lip Stretcher' if mouth_corner_left < upper_lip_y and mouth_corner_right < upper_lip_y else 'None',
        'AU23': 'Lip Tightener' if abs(upper_lip_y - lower_lip_y) < 3 else 'None',
        'AU25': 'Lips Part' if mouth_bottom_y - mouth_top_y > 5 else 'None',
        'AU26': 'Jaw Drop' if mouth_bottom_y - mouth_top_y > 10 else 'None',
        'AU28': 'Lip Suck' if lower_lip_y < upper_lip_y else 'None',
        'AU45': 'Blink' if left_eye_y > nose_tip_y and right_eye_y > nose_tip_y else 'None'
    }

    # Combine AUs with their meanings into a string
    active_aus = [f"{au}: {description}" for au, description in aus.items() if description != 'None']
    return ', '.join(active_aus)

# Function to compute confidence score for a given emotion
def compute_emotion_confidence(active_aus, target_emotion):
    weights = emotion_au_weights.get(target_emotion, {})
    score = sum(weights.get(au, 0) for au in active_aus)
    max_score = sum(weights.values())
    confidence = (score / max_score) * 100 if max_score > 0 else 0
    return confidence

# Function to map AUs to emotions and return both emotion and confidence
def map_aus_to_emotions(aus):
    emotions = {
        'Happiness': 'AU12' in aus and 'AU6' in aus,
        'Sadness': 'AU1' in aus and 'AU4' in aus and 'AU15' in aus,
        'Surprise': 'AU5' in aus and 'AU26' in aus,
        'Disgust': 'AU9' in aus and 'AU10' in aus,
        'Fear': 'AU4' in aus and 'AU5' in aus and 'AU7' in aus,
        'Anger': 'AU4' in aus and 'AU7' in aus,
        'Blinking': 'AU45' in aus
    }

    for emotion, condition in emotions.items():
        if condition:
            confidence = compute_emotion_confidence(aus, emotion)
            return emotion, confidence

    # Return Neutral if no other emotion is detected
    return 'Neutral', 0

# Function to draw blue boxes, green dots, and labels for landmarks
def draw_boxes_and_landmarks(image, face, landmarks, face_count):
    # Draw blue box around the face
    cv2.rectangle(image, (face.left(), face.top()), (face.right(), face.bottom()), (255, 0, 0), 2)

    # Add the face label at the top of the bounding box
    label = f"Face {face_count}"
    cv2.putText(image, label, (face.left(), face.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Draw green dots for facial landmarks
    for i in range(68):
        x = landmarks.part(i).x
        y = landmarks.part(i).y
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

# Function to process images, compute AUs, predict emotions, and save the results in CSV
def process_images_and_save_csv(image_paths, csv_filename='face_aus_emotions_with_confidence.csv'):
    all_face_data = []

    for image_path in image_paths:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        face_count = 1
        image_title = os.path.basename(image_path)

        if len(faces) == 0:
            row = [image_title, 'N/A', 'N/A', 'N/A', 'N/A']
            all_face_data.append(row)
        else:
            for face in faces:
                landmarks = face_landmark_predictor(gray, face)
                draw_boxes_and_landmarks(img, face, landmarks, face_count)

                aus = compute_facs_aus(landmarks)
                active_aus = [au.split(':')[0] for au in aus.split(', ')]

                emotion, confidence = map_aus_to_emotions(active_aus)

                row = [image_title, f"Face {face_count}", aus, emotion, f"{confidence:.2f}%"]
                all_face_data.append(row)
                face_count += 1

        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f'Processed: {image_title}')
        plt.axis('off')
        plt.show()

    columns = ['Image Title', 'Face', 'Active AUs', 'Predicted Emotion', 'Confidence Level']
    df = pd.DataFrame(all_face_data, columns=columns)

    df.to_csv(csv_filename, index=False)
    print(f"Table saved as {csv_filename}")

    return df


# Example usage with 3 images
image_files = [os.path.join(thumbnails_folder, f) for f in os.listdir(thumbnails_folder) if f.endswith(('.png', '.jpg', '.jpeg'))][:3]
df = process_images_and_save_csv(image_files)





Main Code for 1500 images

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import dlib
import pandas as pd
import random

# Folder containing images
thumbnails_folder = r'C:\Users\alexa\Thesis\complete thumbnails\thumbnails'

# Path to the dlib shape predictor
predictor_path = r"C:\Users\alexa\Thesis\shape_predictor_68_face_landmarks.dat\shape_predictor_68_face_landmarks.dat"
face_landmark_predictor = dlib.shape_predictor(predictor_path)

# Initialize dlib's face detector
detector = dlib.get_frontal_face_detector()

# Define AU weights for each emotion to compute confidence levels
emotion_au_weights = {
    'Happiness': {'AU12': 0.5, 'AU6': 0.5},
    'Sadness': {'AU1': 0.3, 'AU4': 0.4, 'AU15': 0.3},
    'Surprise': {'AU5': 0.5, 'AU26': 0.5},
    'Disgust': {'AU9': 0.5, 'AU10': 0.5},
    'Fear': {'AU4': 0.3, 'AU5': 0.3, 'AU7': 0.4},
    'Anger': {'AU4': 0.5, 'AU7': 0.5},
    'Neutral': {}
}

# Function to compute AUs with intensity values (numerical)
def compute_facs_aus(landmarks):
    # Extract key facial coordinates
    left_eye_y = np.mean([landmarks.part(i).y for i in range(36, 42)])  # Left eye
    right_eye_y = np.mean([landmarks.part(i).y for i in range(42, 48)])  # Right eye
    mouth_top_y = np.mean([landmarks.part(i).y for i in range(48, 55)])  # Top of mouth
    mouth_bottom_y = np.mean([landmarks.part(i).y for i in range(55, 60)])  # Bottom of mouth
    left_eyebrow_y = np.mean([landmarks.part(i).y for i in range(17, 22)])  # Left eyebrow
    right_eyebrow_y = np.mean([landmarks.part(i).y for i in range(22, 27)])  # Right eyebrow
    nose_tip_y = landmarks.part(30).y  # Nose tip
    upper_lip_y = landmarks.part(62).y  # Upper lip (central)
    lower_lip_y = landmarks.part(66).y  # Lower lip (central)

    mouth_corner_left = landmarks.part(48).y  # Left corner of mouth
    mouth_corner_right = landmarks.part(54).y  # Right corner of mouth

    # Assigning intensity values for each AU based on facial feature distances
    aus = {
    'AU1': round(max(0, (left_eyebrow_y - left_eye_y) / (left_eye_y - nose_tip_y)), 4),
    'AU2': round(max(0, (left_eyebrow_y - (left_eye_y - 5)) / (left_eye_y - nose_tip_y)), 4),
    'AU4': round(max(0, abs(left_eyebrow_y - right_eyebrow_y) / (left_eye_y - nose_tip_y)), 4),
    'AU5': round(max(0, (left_eye_y - nose_tip_y) / (nose_tip_y - upper_lip_y)), 4),
    'AU6': round(max(0, (mouth_corner_left - mouth_top_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU7': round(max(0, (left_eye_y - nose_tip_y) / (nose_tip_y - upper_lip_y)), 4),
    'AU9': round(max(0, (nose_tip_y - upper_lip_y) / (upper_lip_y - lower_lip_y + 0.0001)), 4),
    'AU10': round(max(0, (upper_lip_y - lower_lip_y) / (mouth_bottom_y - mouth_top_y + 0.0001)), 4),
    'AU12': round(max(0, (mouth_corner_left - mouth_top_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU14': round(max(0, abs(mouth_corner_left - mouth_corner_right) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU15': round(max(0, (mouth_corner_left - upper_lip_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU17': round(max(0, (lower_lip_y - upper_lip_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU20': round(max(0, (mouth_corner_left - upper_lip_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU23': round(max(0, abs(upper_lip_y - lower_lip_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU25': round(max(0, (mouth_bottom_y - mouth_top_y) / (nose_tip_y - upper_lip_y)), 4),
    'AU26': round(max(0, (mouth_bottom_y - mouth_top_y) / (nose_tip_y - upper_lip_y)), 4),
    'AU28': round(max(0, (lower_lip_y - upper_lip_y) / (mouth_bottom_y - mouth_top_y)), 4),
    'AU45': round(max(0, (left_eye_y - nose_tip_y) / (nose_tip_y - upper_lip_y)), 4)
        }


    # Return AUs with their intensity values
    return aus

# Function to compute confidence score for a given emotion
def compute_emotion_confidence(active_aus, target_emotion):
    weights = emotion_au_weights.get(target_emotion, {})
    score = sum(weights.get(au, 0) * active_aus.get(au, 0) for au in active_aus)
    max_score = sum(weights.values())
    confidence = (score / max_score) * 100 if max_score > 0 else 0
    return confidence

# Function to map AUs to emotions and return both emotion and confidence
def map_aus_to_emotions(aus):
    emotions = {
        'Happiness': 'AU12' in aus and 'AU6' in aus,
        'Sadness': 'AU1' in aus and 'AU4' in aus and 'AU15' in aus,
        'Surprise': 'AU5' in aus and 'AU26' in aus,
        'Disgust': 'AU9' in aus and 'AU10' in aus,
        'Fear': 'AU4' in aus and 'AU5' in aus and 'AU7' in aus,
        'Anger': 'AU4' in aus and 'AU7' in aus,
        'Blinking': 'AU45' in aus
    }

    for emotion, condition in emotions.items():
        if condition:
            confidence = compute_emotion_confidence(aus, emotion)
            return emotion, confidence

    # Return Neutral if no other emotion is detected
    return 'Neutral', 0

# Function to draw blue boxes, green dots, and labels for landmarks
def draw_boxes_and_landmarks(image, face, landmarks, face_count):
    # Draw blue box around the face
    cv2.rectangle(image, (face.left(), face.top()), (face.right(), face.bottom()), (255, 0, 0), 2)

    # Add the face label at the top of the bounding box
    label = f"Face {face_count}"
    cv2.putText(image, label, (face.left(), face.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Draw green dots for facial landmarks
    for i in range(68):
        x = landmarks.part(i).x
        y = landmarks.part(i).y
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

# Folder to save processed images
processed_images_folder = r'C:\Users\alexa\Thesis\processed_thumbnails'

# Ensure the folder exists
if not os.path.exists(processed_images_folder):
    os.makedirs(processed_images_folder)

def process_images_and_save_csv(image_paths, csv_filename='face_aus_emotions_with_confidence.csv'):
    all_face_data = []
    
    random.shuffle(image_paths)
    for image_path in image_paths:
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        image_title = os.path.basename(image_path)

        if len(faces) == 0:
            row = [image_title, 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
            all_face_data.append(row)
        else:
            for face in faces:
                # Finding the largest face
                largest_face = max(faces, key=lambda rect: rect.area())
                landmarks = face_landmark_predictor(gray, largest_face)
                draw_boxes_and_landmarks(img, largest_face, landmarks, 1)

                aus = compute_facs_aus(landmarks)
                active_aus = {au: value for au, value in aus.items() if value > 0}

                emotion, confidence = map_aus_to_emotions(active_aus)

                left_eye_coords = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
                right_eye_coords = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]
                nose_coords = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(27, 36)]
                mouth_coords = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)]

                row = [image_title, aus, emotion, f"{confidence:.2f}%",
                       left_eye_coords, right_eye_coords, nose_coords, mouth_coords]
                all_face_data.append(row)

            # Save processed image
            processed_image_path = os.path.join(processed_images_folder, image_title)
            cv2.imwrite(processed_image_path, img)

        df = pd.DataFrame(all_face_data, columns=['Image', 'AUs', 'Emotion', 'Confidence',
                                              'Left Eye', 'Right Eye', 'Nose', 'Mouth'])
        df.to_csv(csv_filename, index=False)



        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f'Processed: {image_title}')
        plt.axis('off')
        plt.show()

    columns = ['Image Title', 'Active AUs', 'Predicted Emotion', 'Confidence Level',
               'Left Eye Coordinates', 'Right Eye Coordinates', 'Nose Coordinates', 'Mouth Coordinates']
    df = pd.DataFrame(all_face_data, columns=columns)

    df.to_csv(csv_filename, index=False)
    print(f"Table saved as {csv_filename}")

    return df


# Example usage with 3k images
image_files = [os.path.join(thumbnails_folder, f) for f in os.listdir(thumbnails_folder) if f.endswith(('.png', '.jpg', '.jpeg'))][:1500]
df = process_images_and_save_csv(image_files)


In [ ]:
# Folder to save processed images

processed_folder = r'C:\Users\alexa\Thesis\processed_thumbnails'
os.makedirs(processed_folder, exist_ok=True)

def process_images_and_save_csv(image_paths, csv_filename='face_aus_emotions_with_confidence.csv'):
    all_face_data = []

    print(f"Processing {len(image_paths)} images...")  # Debugging

    for image_path in image_paths:
        print(f"Processing: {image_path}")

        img = cv2.imread(image_path)

        if img is None:
            print(f" Failed to load image: {image_path}")
            continue  # Skip this image

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        image_title = os.path.basename(image_path)

        if len(faces) == 0:
            print(f"⚠️ No faces detected in {image_title}")
            row = [image_title, 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
            all_face_data.append(row)
        else:
            largest_face = max(faces, key=lambda rect: rect.area())
            landmarks = face_landmark_predictor(gray, largest_face)
            draw_boxes_and_landmarks(img, largest_face, landmarks, 1)

            aus = compute_facs_aus(landmarks)
            active_aus = {au: value for au, value in aus.items() if value > 0}
            emotion, confidence = map_aus_to_emotions(active_aus)

            row = [image_title, aus, emotion, f"{confidence:.2f}%"]
            all_face_data.append(row)

            processed_image_path = os.path.join(processed_folder, image_title)
            success = cv2.imwrite(processed_image_path, img)

            if success:
                print(f"✅ Saved: {processed_image_path}")
            else:
                print(f" Failed to save: {processed_image_path}")

    print("Processing complete.")
